In [91]:
import requests
import pandas as pd
from datetime import datetime
import ccxt
import sqlite3
import config


In [92]:
ex = 'indodax' 
currency_f = 'idr'
currency_c = 'eth'
pair = currency_c.upper() + '/' +currency_f.upper()
now = datetime.utcnow()
now_ts = now.timestamp()

In [93]:
def get_fx(currency_f):
    fx = requests.get('https://apilayer.net/api/live?access_key=a1d6d82a3df7cf7882c9dd2b35146d6e&source=USD&format=1').json()
    return fx['quotes']['USD' + currency_f.upper()]

In [94]:
arg = {
    'apiKey': config.indodax['apiKey'],
    'secret': config.indodax['secret']
}

params = {
    'pair': 'eth_idr',
    'count': 50, # number of trades return 
    'end': now_ts
}



In [95]:
def get_td(ex):
    ex_instance = eval('ccxt.' + ex)(arg)
    trades = ex_instance.privatePostTradeHistory(params)['return']['trades']
    df = pd.DataFrame(trades)
    df['time'] = df['trade_time'].apply(lambda x:datetime.fromtimestamp(int(x)))
    df = df.rename(columns = {'trade_time':'utc'})
    return df

In [97]:
df = get_td(ex)
df.head()

,trade_id,order_id,type,eth,price,fee,utc,time
0,2703744,24630771,sell,0.10077594,2968000,0,1587804322,2020-04-25 09:45:22
1,2703743,24630735,sell,0.02444291,2969000,0,1587804286,2020-04-25 09:44:46
2,2703717,24630445,sell,0.01679050,2969000,0,1587803288,2020-04-25 09:28:08
3,2703700,24630285,sell,0.00368920,2973000,0,1587802955,2020-04-25 09:22:35
4,2703697,24630263,sell,0.03520040,2974000,0,1587802844,2020-04-25 09:20:44


In [90]:
# connect to sql
conn = sqlite3.connect('mysqlite.db')
table_name = "td_" + ex 

# append
df.to_sql(table_name, conn, if_exists = 'append')

# select unique entry in pandas 
df_pre = pd.read_sql_query("SELECT * from {}".format(table_name),conn)
df_new = df_pre.drop_duplicates(subset = ["trade_id"])

# replace the previous sql
df_new.to_sql(table_name, conn, if_exists = 'replace', index_label='id')
# check 
pd.read_sql_query("SELECT * from {}".format(table_name),conn)


,id,index,trade_id,order_id,type,eth,price,fee,utc,time
0,0,0,2703744,24630771,sell,0.10077594,2968000,0,1587804322,2020-04-25 09:45:22
1,1,1,2703743,24630735,sell,0.02444291,2969000,0,1587804286,2020-04-25 09:44:46
2,2,2,2703717,24630445,sell,0.01679050,2969000,0,1587803288,2020-04-25 09:28:08
3,3,3,2703700,24630285,sell,0.00368920,2973000,0,1587802955,2020-04-25 09:22:35
4,4,4,2703697,24630263,sell,0.03520040,2974000,0,1587802844,2020-04-25 09:20:44
...,...,...,...,...,...,...,...,...,...,...
95,95,95,2700009,24582581,buy,1.47002135,2894000,0,1587653255,2020-04-23 15:47:35
96,96,96,2699984,24582498,buy,4.78969975,2891000,0,1587653187,2020-04-23 15:46:27
97,97,97,2699968,24582395,buy,5.43719972,2894000,0,1587653027,2020-04-23 15:43:47
98,98,98,2699905,24581882,buy,0.12121153,2906000,0,1587652407,2020-04-23 15:33:27


In [80]:
conn.close()